<center>
<a href="https://github.com/kamu-data/kamu-cli">
<img alt="kamu" src="https://raw.githubusercontent.com/kamu-data/kamu-cli/master/docs/readme_files/kamu_logo.png" width=270/>
</a>
</center>

<br/>

<div align="center">
<a href="https://github.com/kamu-data/kamu-cli">Repo</a> | 
<a href="https://docs.kamu.dev/cli/">Docs</a> | 
<a href="https://docs.kamu.dev/cli/learn/learning-materials/">Tutorials</a> | 
<a href="https://docs.kamu.dev/cli/learn/examples/">Examples</a> |
<a href="https://docs.kamu.dev/cli/get-started/faq/">FAQ</a> |
<a href="https://discord.gg/nU6TXRQNXC">Discord</a> |
<a href="https://kamu.dev">Website</a>
</div>


<center>

<br/>
    
# Covid Jupyter Notebook

</center>

In this tutorial there is a step by step guide on how to use the Covid data make visually pleasing graphs and use SQL to edit data.

<div class="alert alert-block alert-success">
To follow this example checkout kamu-cli repository and navigate into the examples/covid sub-directory.

Create a temporary kamu workspace in that folder using:
<p style="background:black">
<code style="background:black;color:white">kamu init
</code>
</p>
</div>

<div class="alert alert-block alert-success">
Then add all dataset manifests found in the current directory:
<p style="background:black">
<code style="background:black;color:white">kamu add --recursive .
kamu pull --all
</code>
</p>
</div>

## Load Kamu Extension
<div class="alert alert-block alert-success">
Start by loading <code>kamu</code> Jupyter extension in your terminal:
</div>

In [1]:
%load_ext kamu

## Import and Test Data
<div class="alert alert-block alert-success">
Now it is time to start importing your Covid data by province. First import the data from the province of BC by using the command <code>%import dataset</code>. An alias was created to make it easier to call this file.
</div>

In [2]:
%import_dataset covid19.british-columbia.case-details --alias cases_bc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
To test if the data was loaded correctly a SQL querry is run.
</div>

In [3]:
%%sql
SELECT * FROM cases_bc
ORDER BY id DESC
LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
Now it is time to import the rest of the Covid data files and create aliases for them
</div>

In [4]:
%import_dataset covid19.ontario.case-details --alias cases_on
%import_dataset covid19.alberta.case-details --alias cases_ab

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
Time to test again if the data was imported correctly. You can test the Alberta files by changing  <code>cases_on</code> to   <code>cases_ab</code>. 
</div>

In [5]:
%%sql
SELECT * FROM cases_on
ORDER BY id DESC
LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
Now we are importing a yaml file which allows for changing the age ranges of each of the three provinces so that they align  with eachother. For example only one province has a 90+ whereas the other two has age ranges of 80+. Therefore we need to switch the age ranges to 80+ to compare the data.
</div>

In [6]:
%import_dataset abbcon.case-details --alias cases_three

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
Time to test again if the data was imported correctly.
</div>

In [7]:
%%sql
SELECT * FROM cases_three
LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">With the below SQL querry you can compare the cases per age range per province
 </div>

In [8]:
%%sql
SELECT province, age_group, COUNT(*) 
FROM cases_three
GROUP BY province, age_group
ORDER BY province, age_group;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
Since Quebec does not show data by age, we are importing this data now. However, if you want to test whether the data is correct, you would repeat the SQL query used for case_bc and case_on, but change id to row_id since Quebec uses different headers. This causes issues when combining data which will be talked about more later.
</div>

In [9]:
%import_dataset covid19.quebec.case-details --alias cases_qb

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
The next file that you import is case details for the four provinces combined. The file <code>covid19.canada.case-details</code>  uses an SQL query in the yaml file to combine that data so that you don't have to combine them.
The SQL data can be found in this yaml. If you open this yaml, you will see an sql query to change the headers to combine the data.
</div>

In [10]:
%import_dataset covid19.canada.case-details --alias cases_four_provinces

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
Again there is a test to see if the data worked by showing the last 10 data rows.
</div>

In [11]:
%%sql
SELECT * FROM cases_four_provinces
LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
To use this file, a SQL query is created to combine all of the cases by gender and by province
</div>

In [12]:
%%sql -o total_cases
SELECT province, gender, COUNT(*) 
FROM cases_four_provinces
GROUP BY province, gender
ORDER BY province, gender;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
    Through <code>plotly.express.bar</code> a bar chart can be created to compare the cases per province then per gender (male, female, unspecified).
</div>

In [13]:
%%local 
import plotly.express 
plotly.express.bar(total_cases, x='province', y='count(1)', color='gender', title='Cases per Gender')


<div class="alert alert-block alert-info">
    By looking through the data you can see that Quebec has a large amount of people who were classified as undefined. This is probably due to Quebec's strict privacy act laws that are part of the Act Respecting Access to Documents Held by Public Bodies and the Protection of Personal Information
</div>

<div class="alert alert-block alert-success">
The last dataset that we are importing is daily cases for the four provinces.
</div>

In [14]:
%import_dataset covid19.canada.daily-cases --alias daily_cases

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-success">
Now test again to see if the data was succcesfully installed for this file.
</div>

In [15]:
%%sql -o daily_cases
select * from daily_cases

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

<div class="alert alert-block alert-success">
The last step is to create a line plot graph to compare the different amount of cases per day by province.
</div>

In [16]:
%%local
import plotly.express
plotly.express.line(daily_cases, x="reported_date" , y="total_daily", color="province")

<div class="alert alert-block alert-info">
As seen in the graph above, there are two significant spikes in Quebec's COVID-19 data from late December 2020 and early January 2021. These spikes likely don't reflect an actual surge in cases but rather a delay in data entry due to the holiday weekend. In Kamu, we use watermarks, which are markers that indicate when we have probably received all relevant data for a particular period. Given the potential for delays like these, the watermark can be adjusted to account for late-arriving data, ensuring that all reports are accurately included in the analysis.
</div>